In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments,DataCollatorForSeq2Seq
import torch
from torch.utils.data import Dataset
import pandas as pd

In [ ]:
train_path='/kaggle/input/coldata/train.csv'
dev_path='/kaggle/input/coldata/dev.csv'
save_path='/kaggle/working/bert'

In [ ]:
args = TrainingArguments(
    output_dir=save_path,
    per_device_train_batch_size=40,
    gradient_accumulation_steps=2,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=1000,
    learning_rate=1e-7,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

In [ ]:
class MyData(Dataset):
    def __init__(self,path):
        super(MyData,self).__init__()
        self.input=[]
        self.output=[]
        train_data=pd.read_csv(path)
        for i in range(len(train_data['label'])): 
            self.input.append(tokenizer(train_data['TEXT'][i], padding="max_length", max_length=256))
            self.output.append(train_data['label'][i])
    def __len__(self):
        return len(self.input)
    def __getitem__(self, index): 
        return dict(
            input_ids=self.input[index]['input_ids'],
            attention_mask=self.input[index]['attention_mask'],
            label=self.output[index],
        )

In [ ]:
train_dataset=MyData(train_path)
eval_dataset=MyData(dev_path)

In [ ]:
print(train_dataset[1])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()

In [47]:
text = "哈哈哈哈哈天津人听了想打人哈哈哈哈哈哈"
encoded_input = tokenizer(text, return_tensors='pt').to('cuda')
output = model(**encoded_input)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4390, -2.0650]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
